# 전문가를 위한 파이썬

## part 2

## 챕터2

### 시퀀스
* 컨테이너 시퀀스
    - 서로 다른 자료형의 항목들을 담을 수 있는 list, tuple, collections.deque형
* 균일 시퀀스
    - 단 하나의 자료형만을 담을 수 있는 str, bytes, bytearray, memoryview, array.array형
* 가변 시퀀스
    - list, bytearray, array.array, collections.deque, memoryview형
* 불변 시퀀스
    - tuple, str, bytes형

### 지능형 시퀀스
* 지능형 리스트나 제너레이터 표현식을 사용하면 시퀀스를 간단히 생성할 수 있다. 이러한 코드들은 가독성이 좋고 때로는 실행 속도도 빠른 코드를 만들 수 있다.

In [1]:
#ver1
symbols='!@#$%^&*'
codes=[]
for symbol in symbols:
    codes.append(ord(symbol))
print(codes)

[33, 64, 35, 36, 37, 94, 38, 42]


In [2]:
#ver2
symbols='!@#$%^&*'
codes=[ord(symbol) for symbol in symbols]
print(codes)

[33, 64, 35, 36, 37, 94, 38, 42]


* 위의 두 버전의 코드는 같은 역할을 하지만 그 길이나 코드의 의도가 확실하게 드러나는 쪽은 ver2쪽이다.

* 파이썬 3버전 부터는 지능형 리스트, 제너레이터 표현식, 그리고 이와 동급인 지능형 집합과 지능협 딕셔너리는 함수터럼 고유한 지역 범위를 가져 메모르 누수를 발생시키지 않는다.

* map()고 filter()함수를 이용해서 수행할 수 있는 작업은 기능적으로 문제가 있는 파이썬 lambdaa를 억지로 끼워 넣지 않고도 지능형 리스트를 이용해서 모두 구현 가능하다.

In [3]:
#ver1
symbols='!@#$%^&*'
beyond_ascii=[ord(s) for s in symbols if ord(s)>40]
print(beyond_ascii)

[64, 94, 42]


In [4]:
#ver2
symbols='!@#$%^&*'
beyond_ascii=list(filter(lambda c:c>40, map(ord, symbols)))
print(beyond_ascii)

[64, 94, 42]


* 위의 두 버전의 코드도 같은 역할을 하지만 ver2는 lambda를 사용하지 않는다.
* lambda에 기능적인 문제가 있다고 하는 이유 (https://youngwonhan-family.tistory.com/5)
    - TraceBack 결과로 부터 어떤 함수에서 발생했는지 알 수 없다.
    - lambda는 이름이 없는 익명함수로 존재한다. 이름이 없으므로 오류가 발생한 경우 함수 추적이 불가하다.

### 튜플 언패킹

In [10]:
for a, *b in [(1, 2, 3), (4, 5, 6, 7)]:
    print(a, b)

1 [2, 3]
4 [5, 6, 7]


In [4]:
a, *b, c, d=[1, 2, 3, 4, 5]
print(a)
print(b)
print(c)
print(d)

1
[2, 3]
4
5


In [9]:
a, *b, c, d=range(1, 6)
print(a)
print(b)
print(c)
print(d)

1
[2, 3]
4
5


* 위의 방식과도 같은 언팩킹도 가능하다.
* 참고할만한 추가사항 [https://www.python.org/dev/peps/pep-3132/]

### 튜플은 단순한 불변 리스트가 아니다
* namedtuple에 대해서는 위에서 설명했다.

* 튜플은 과연 불변인가?

In [5]:
try:
    t=(1,2,[3,4])
    t[2]+=[5,6]
except:
    print("error!!")
 
print(t)

error!!
(1, 2, [3, 4, 5, 6])


* 위와 같은 코드에서 t[2]에 [5, 6]을 더하는 행위는 분명 에러를 발생시킨다. 하지만 tuple의 값을 변했음을 알 수 있다.
    - 이는 tuple이 완전 불변이 아님을 알 수 있고 아래의 상황을 조심해야 한다.
    - 1. 가변 객체를 튜블과 같은 불변 객체에 넣는 것을 좋은 생각이 아니다.
    - 2. 복합 할당은 원자적인 연상이 아니다.(아래의 예제에서 일부 연산이 수행된 후 예외가 발생했다.)
    - 3. 파이썬 바이트코드를 살펴보는 것은 그리 어렵지 않으며 내부에서 어떤 일이 발생하고 있는지 살펴보는 데 도움이 된다.

In [6]:
import dis
dis.dis('s[a]+=b')

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


### 시퀀스의 복합 할당

* ```+=``` 과 ```*=``` 등의 복합 할당 연산자를 작동하도록 하는 특수 메서드는 ```__iadd__()``` 와 ```__imul()__``` 이다. 그런데 이 메서드가 구현되어있지 않으면 파이썬은 대신 ```__add__()``` 와 ```__mul__()``` 메서드를 호출한다.
* ```__iadd__()``` 의 구현 여부에 따라 ```a+=b``` 와 같은 연산에서 a 변수가 가리키는 객체의 정체성이 바뀔수도 있고 바뀌지 않을 수도 있다.
* 일반적으로 가변 시퀀스에 대해서는 ```__iadd__()``` 메서드를 구현해서 += 연산자가 기존 객체의 내용을 변경하게 만드는 것이 좋다.
* 불변 시퀀스에 경우에는 이 연산을 수행할 수 없다.

* 아래는 ```*=``` 연산자를 가변 시퀀스와 불변 시퀀스에 적용한 예이다.

In [16]:
l=[1, 2, 3]
print(id(l))
l*=2
print(l)
print(id(l))

# 리스트의 경우 초기 리스트의 id와 연산 후 리스트의 id가 같다. 즉 같은 객체이다.

2335173769224
[1, 2, 3, 1, 2, 3]
2335173769224


In [15]:
t=(1, 2, 3)
print(id(t))
t*=2
print(t)
print(id(t))

# 튜플의 경우 초기 튜플의 id와 연산 후 튜플의 id가 다르다. 즉 연산 후 새로운 객체가 만들어졌다.

2335173512648
(1, 2, 3, 1, 2, 3)
2335173551016


### list.sort()와 sorted() 내장 함수
* list.sort() 메서드는 사본을 만들지 않고 리스트 내부를 변경해서 정렬한다. 때문에 sort()메서드는 타깃 객체를 변경하고 새로운 리스트를 생성하지 않았음을 알려주기 위해 None을 반환한다.

* 이와는 반대로 sorted() 내장 함수는 새로운 리스트를 생성해서 반환한다.

* 아래의 예제를 참고

In [8]:
fruits=['grape', 'raspberry', 'apple', 'banana']
print(sorted(fruits))
print(fruits)
print(sorted(fruits, reverse=True))
print(sorted(fruits, key=len))
print(sorted(fruits, key=len, reverse=True))
print(fruits)
print(fruits.sort())
print(fruits)

['apple', 'banana', 'grape', 'raspberry']
['grape', 'raspberry', 'apple', 'banana']
['raspberry', 'grape', 'banana', 'apple']
['grape', 'apple', 'banana', 'raspberry']
['raspberry', 'banana', 'grape', 'apple']
['grape', 'raspberry', 'apple', 'banana']
None
['apple', 'banana', 'grape', 'raspberry']


### bisect 모듈

* 위 모듈은 ```bisect()``` 과 ```insort()``` 함수를 제공한다.
    * ```bisect()``` 는 이진 검색 알고리즘을 이용해서 시퀀스를 검색한다.
    * ```insert()``` 는 정렬된 시퀀스 안에 항목을 삽입한다.

In [20]:
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d} {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)
        offset = position * ' |'
        print(ROW_FMT.format(needle, position, offset))
            
if __name__ == '__main__':
    if sys.argv[-1] == 'left':
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect
    print('DEMO:', bisect_fn.__name__)
    print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
    demo(bisect_fn)


DEMO: bisect_right
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14  | | | | | | | | | | | | | |31
30 @ 14  | | | | | | | | | | | | | |30
29 @ 13  | | | | | | | | | | | | |29
23 @ 11  | | | | | | | | | | |23
22 @  9  | | | | | | | | |22
10 @  5  | | | | |10
 8 @  5  | | | | |8 
 5 @  3  | | |5 
 2 @  1  |2 
 1 @  1  |1 
 0 @  0 0 


* 정렬 연산을 계속 하는것을 효율적이지 못하기 때문에 정렬된 상태를 유지하는 것이 좋다. 때문에 ```insort()``` 가 만들어졌다.
* ```insort(seq, item)``` 은 ```seq``` 를 오른차순으로 유지한체 ```item``` 을 ```seq``` 에 삽입한다.

In [21]:
import bisect
import random

SIZE = 7

random.seed(1729)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE*2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


### 리스트가 답이 아닐 때

* 리스트형은 융통성 있고 사용하기 편하지만, 세부 요구사항에 따라 더 나은 자료형도 있다.
* 배열이나 메모리 뷰, NumPy, SciPy, queue나 deque등 많은 자료형이 있으니 세부 요구사항에 대해서 잘 생각해보고 자료형을 선택해서 사용하도록 하자.